In [1]:
import pickle
import os
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec
import gensim
import numpy as np
import pandas as pd
from random import shuffle
import preprocessor_end
import helper_functions

C:\Users\Abdo\PycharmProjects\final_project_1\venv\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
folder_path = "C:/Users/Abdo/Project/data/subtitles/"
folders_names = os.listdir(folder_path)

In [4]:
# data_dictionary
# has three keys (0_label, 1_label, no_label)
# values are dictionaries that has as keys the names of files and as values the preprocessed corresponding file 

data_dictionary = {
    "0_label":{},
    "1_label":{},
    "no_label":{}
}

In [ ]:
'''for folder_name in folders_names:
    dict = data_dictionary[folder_name]
    files = os.listdir(folder_path+folder_name+"/")
    for file in files:
        if(file.endswith(".srt")):
            dict[file] = preprocessor_end.preprocess_file(folder_path+folder_name+"/", file, False)
        else:
            dict[file] = preprocessor_end.preprocess_file(folder_path+folder_name+"/", file, True)'''

In [27]:
#with open('C:/Users/Abdo/Project/pickle_files/data_dictionary_before_removing_frequent_words.pkl', 'wb') as output:
    #pickle.dump(data_dictionary, output)

In [3]:
pickle_in = open('C:/Users/Abdo/Project/pickle_files/data_dictionary_before_removing_frequent_words.pkl','rb')
data_dictionary = pickle.load(pickle_in)

In [10]:
# all the texts (arrays of words) combined in a single array (array of arrays)
all_texts = []
for label in data_dictionary.keys():
    for file_name in data_dictionary[label].keys():
        all_texts.append(data_dictionary[label][file_name])

In [11]:
model = gensim.models.Word2Vec(
        all_texts,
        size=150,
        window=10,
        min_count=5,
        workers=10,iter=10,sample=0.01)

In [12]:
model.train(all_texts, total_examples=len(all_texts), epochs=10)

(220806900, 222669120)

In [13]:
#with open('C:/Users/Abdo/Project/pickle_files/word2vec_model_trained_150.pkl', 'wb') as output:
#pickle.dump(model, output)

In [5]:
pickle_in = open('C:/Users/Abdo/Project/pickle_files/word2vec_model_trained_150.pkl','rb')
model = pickle.load(pickle_in) # model trained on our data set

In [14]:
vocabulary = set(model.wv.index2word)

In [11]:
# converts the glove file to word2vec file, then load the word2vec model and returns it 
def glove_file_to_word2vec_model(glove_file_path):
    glove_file = datapath(glove_file_path)
    tmp_file = get_tmpfile("word2vec.txt")
    
    # call glove2word2vec script
    # default way (through CLI): python -m gensim.scripts.glove2word2vec --input <glove_file> --output <w2v_file>
    glove2word2vec(glove_file, tmp_file)
    return KeyedVectors.load_word2vec_format(tmp_file)


In [103]:
#word2vec_glove_model = glove_file_to_word2vec_model('C:/Users/Abdo/Project/glove.840B.300d.txt')

In [104]:
#with open('word2vec models/word2vecModel_glove_840b_300d.pkl', 'wb') as output:
    #pickle.dump(word2vec_glove_model, output)

In [ ]:
#pickle_in = open('word2vec models/word2vecModel_glove_6b_200d.pkl','rb')
#word2vec_glove_model = pickle.load(pickle_in)

In [15]:
# returns the average, n dimensioned, vector embedding of every word in a given document
# if a given word is not in the vocabulary of the model, it igonores it
def average_word_vectors(words, model, vocabulary, num_features):
    feature_vector = np.zeros((num_features,),dtype="float64")
    nwords = 0.
    for word in words:
        if word in vocabulary: 
            nwords = nwords + 1.
            feature_vector = np.add(feature_vector, model[word])
    
    if nwords:
        feature_vector = np.divide(feature_vector, nwords)
        
    return feature_vector

In [16]:
# has two keys (0_label and 1_label) with a list of 150d vectors for each text
featureVector_label_dictionary = {"0_label":[], "1_label":[]}

In [17]:
for label in list(data_dictionary.keys())[0:2]:
    for file_name in data_dictionary[label].keys():
        text_feature_vector = average_word_vectors(data_dictionary[label][file_name], model, vocabulary, 150)
        featureVector_label_dictionary[label].append(text_feature_vector)

C:\Users\Abdo\PycharmProjects\final_project_1\venv\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if __name__ == '__main__':


In [18]:
#with open('C:/Users/Abdo/Project/pickle_files/featureVector_label_dictionary.pkl', 'wb') as output:
    #pickle.dump(featureVector_label_dictionary, output)

In [8]:
pickle_in = open('C:/Users/Abdo/Project/pickle_files/featureVector_label_dictionary.pkl','rb')
featureVector_label_dictionary = pickle.load(pickle_in)

In [11]:
# data in tuples : list of tuples : [(text_vector, label),.......]
textVector_label_pairs = []

In [12]:
for label in featureVector_label_dictionary.keys():
    for vector in featureVector_label_dictionary[label][0:1581]:
        textVector_label_pairs.append((vector, label))

In [13]:
shuffle(textVector_label_pairs)
shuffle(textVector_label_pairs)

In [14]:
x = []
y = []

for data_pair in textVector_label_pairs:
    x.append(data_pair[0])
    y.append(data_pair[1])

In [15]:
from sklearn import svm
clf = svm.SVC(gamma=0.001,C=100)

In [16]:
clf.fit(x[:2000],y[:2000])

SVC(C=100, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [17]:
print(clf.score(x[2000:],y[2000:]))

0.8098106712564543


In [18]:
#with open('C:/Users/Abdo/Project/pickle_files/svm_model.pkl', 'wb') as output:
    #pickle.dump(clf, output)

In [ ]:
#pickle_in = open('C:/Users/Abdo/Project/pickle_files/svm_model.pkl','rb')
#clf = pickle.load(pickle_in)

In [29]:
def test_model(word2vec_model, ml_model, vocabulary ,num_of_features, text_directory_path, text_name):
    tokenized_text = preprocessor_end.preprocess_file(text_directory_path,text_name, True)
    text_vector = average_word_vectors(tokenized_text, word2vec_model, vocabulary, num_of_features)
    print( ml_model.predict([text_vector]) )

In [30]:
test_model(model, clf, vocabulary, 150, "C:/Users/Abdo/Desktop/","Moana.2016.BDRip.x264-GECKOS + YTS.AG.srt")

Moana.2016.BDRip.x264-GECKOS + YTS.AG.srt


['0_label']


C:\Users\Abdo\PycharmProjects\final_project_1\venv\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if __name__ == '__main__':
